In [43]:
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
import sqlite3
from datetime import datetime, timedelta
import requests
import pandas_market_calendars as mcal

# Function to calculate a triangular moving average
def triangular_moving_average(series, n):
    # Calculate the triangular moving average with a two-step rolling mean
    smoothed_series = series.rolling(window=(n // 2), min_periods=1).mean()
    smoothed_series = smoothed_series.rolling(window=(n // 2), min_periods=1).mean()
    return smoothed_series

# Function to add Short Term Triangular Moving Averages and Indicators
def add_triangular_moving_averages_and_indicators(data):
    # Calculate 30-day and 60-day Triangular Moving Averages
    data['30_TMA'] = triangular_moving_average(data['Adj_Close'], 30)
    data['60_TMA'] = triangular_moving_average(data['Adj_Close'], 60)
    
    # Define 30-Day and 60-Day Indicators
    data['30_Day_Indicator'] = np.where(data['Adj_Close'] > data['30_TMA'], 'Bullish', 'Bearish')
    data['60_Day_Indicator'] = np.where(data['Adj_Close'] > data['60_TMA'], 'Bullish', 'Bearish')
    
    # Shift the indicators by 1 day to use previous day's indicators for today
    data['30_Day_Indicator'] = data['30_Day_Indicator'].shift(1)
    data['60_Day_Indicator'] = data['60_Day_Indicator'].shift(1)
    
    return data

# Function to define market regimes
def define_market_regimes(data):
    # Calculate 250-day triangular moving average
    data['250_TMA'] = triangular_moving_average(data['Adj_Close'], 250)
    
    # Define the four market regimes for 250 TMA
    conditions = [
        (data['Vol_Regime'] == 1) & (data['Adj_Close'] < data['250_TMA']),
        (data['Vol_Regime'] == 1) & (data['Adj_Close'] >= data['250_TMA']),
        (data['Vol_Regime'] == 0) & (data['Adj_Close'] < data['250_TMA']),
        (data['Vol_Regime'] == 0) & (data['Adj_Close'] >= data['250_TMA']),
    ]
    choices = [
        'Bearish High Variance',
        'Bullish High Variance',
        'Bearish Low Variance',
        'Bullish Low Variance'
    ]
    
    # Specify a default value that matches the data type of choices
    data['Market_Regime'] = np.select(conditions, choices, default='Unknown')
    
    # Shift the market regime by 1 day to use previous day's regime for today
    data['Adjusted_Market_Regime'] = data['Market_Regime'].shift(1)
    
    return data

# Function to calculate exposures
def calculate_exposures(data):
    # Define initial exposure based on Adjusted_Market_Regime
    exposure_mapping = {
        'Bullish Low Variance': 2.0,
        'Bearish Low Variance': 1.0,
        'Bullish High Variance': 1.0,
        'Bearish High Variance': 0.0
    }
    data['Portfolio_Exposure'] = data['Adjusted_Market_Regime'].map(exposure_mapping).fillna(1.0)  # Default exposure is 1.0 if regime is NaN
    
    # Adjust exposure based on 30-Day and 60-Day Indicators
    for index, row in data.iterrows():
        if row['Portfolio_Exposure'] == 2.0:
            if row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bearish':
                data.at[index, 'Portfolio_Exposure'] = 1.0
            elif row['30_Day_Indicator'] == 'Bullish' and row['60_Day_Indicator'] == 'Bearish':
                data.at[index, 'Portfolio_Exposure'] = 1.5
            elif row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bullish':
                data.at[index, 'Portfolio_Exposure'] = 1.5
                        
        # Adjust exposure based on 30-Day and 60-Day Indicators for exposure = 1.0 and Bearish Low Variance regime
        if row['Portfolio_Exposure'] == 1.0 and row['Adjusted_Market_Regime'] == 'Bearish Low Variance':
            if row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bearish':
                data.at[index, 'Portfolio_Exposure'] = 0.0
            elif row['30_Day_Indicator'] == 'Bullish' and row['60_Day_Indicator'] == 'Bearish':
                data.at[index, 'Portfolio_Exposure'] = 1.0
            elif row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bullish':
                data.at[index, 'Portfolio_Exposure'] = 1.0
                        
    return data

# Function to send Telegram message
def telegram_messenger():
    # Telegram Bot API token and Channel ID
    bot_token = '7328648943:AAH3gHyGf2xgjxBfzPd05F_7IagASgs-Dj0'
    channel_id = '-1002309744206'

    # Connect to the database
    conn = sqlite3.connect('financial_model_test.db')

    # Query the last two rows from 'Adjusted_Market_Regime' and 'Date' columns
    query = """
    SELECT Date, Adjusted_Market_Regime, Portfolio_Exposure 
    FROM financial_data 
    ORDER BY DATE(Date) DESC 
    LIMIT 2
    """

    # Execute query and load into a DataFrame
    data = pd.read_sql_query(query, conn)

    # Close the database connection
    conn.close()

    # Convert 'Date' to datetime
    data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')

    # Sort data by Date in ascending order
    data = data.sort_values('Date').reset_index(drop=True)

    # Debugging: Print the data used for the Telegram message
    print("Data used for Telegram message:")
    print(data)

    # Initialize the message variable each time the code runs with bold header
    message = "<b>Your Daily Portfolio Exposure Update</b>\n\n"  # Reset message here
    labels = ["Today's Market Regime", "Tomorrow's Market Regime"]

    # Loop through the DataFrame and format the message
    for index, row in data.iterrows():
        # Format Date
        formatted_date = row['Date'].strftime('%m/%d/%Y').lstrip("0").replace("/0", "/")

        # Add the labeled message for each row with line breaks for better formatting
        message += f"<u>{labels[index]}</u>\n"
        message += f"<i>Date</i>: {formatted_date}\n"
        message += f"<i>Adjusted Market Regime</i>: {row['Adjusted_Market_Regime']}\n"
        message += f"<i>Portfolio Exposure</i>: {row['Portfolio_Exposure'] * 100:.0f}%\n\n" # Format Portfolio_Exposure as a percentage

    # Telegram API URL
    api_url = f'https://api.telegram.org/bot{bot_token}/sendMessage'

    # Payload to send with HTML formatting enabled
    payload = {
        'chat_id': channel_id,
        'text': message,
        'parse_mode': 'HTML'  # Enables HTML for bold formatting
    }

    # Send the request
    response = requests.post(api_url, json=payload)

    # Check the response
    if response.status_code == 200:
        print('Message sent successfully!')
    else:
        print(f'Failed to send message. Error: {response.text}')

if __name__ == "__main__":
    # Parameters
    ticker = '^GSPC'
    start_date = '1950-01-01'
    end_date = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
    # Download data
    data = yf.download(ticker, start=start_date, end=end_date)

    # Use adjusted close price
    data = data[['Adj Close']]
    data.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)

    # Ensure that the index is timezone-naive
    data.index = data.index.tz_localize(None)

    # Calculate daily returns
    data['Index_Returns'] = data['Adj_Close'].pct_change()
    data.dropna(inplace=True)

    # Prepare returns for the model
    returns = data['Index_Returns']

    # Fit Markov Switching Model
    model = MarkovRegression(returns, k_regimes=2, trend='c', switching_variance=True)
    result = model.fit()
    print(result.summary())

    # Add regime to the data
    data['Vol_Regime'] = result.smoothed_marginal_probabilities.idxmax(axis=1)

    # Predict the next day's volatility regime
    # Extract smoothed probabilities and last known state probabilities
    smoothed_probs = result.smoothed_marginal_probabilities
    last_probs = smoothed_probs.iloc[-1].values

    # Extract transition probabilities from the model parameters
    params = result.params
    p_00 = params['p[0->0]']
    p_10 = params['p[1->0]']
    p_01 = 1 - p_00
    p_11 = 1 - p_10

    # Construct the transition matrix
    transition_matrix = np.array([
        [p_00, p_01],
        [p_10, p_11]
    ])

    # Update state probabilities to predict the next day's regime (t+1)
    state_probs_t1 = np.dot(last_probs, transition_matrix)

    # Determine the most likely regime at t+1
    regime_labels = smoothed_probs.columns.tolist()  # Should be [0, 1]
    most_likely_regime_t1 = regime_labels[np.argmax(state_probs_t1)]

    # Get the next trading day using the NYSE calendar
    nyse = mcal.get_calendar('NYSE')
    last_date = data.index[-1]

    # Create a schedule that covers the next few days
    schedule = nyse.schedule(start_date=last_date, end_date=last_date + timedelta(days=10))

    # Remove timezone information from schedule.index
    schedule.index = schedule.index.tz_localize(None)

    # Find the next trading day after 'last_date'
    next_trading_day_index = schedule.index.searchsorted(last_date, side='right')

    if next_trading_day_index < len(schedule.index):
        next_trading_day = schedule.index[next_trading_day_index]
    else:
        # If there is no next trading day in the schedule, extend the schedule and try again
        extended_schedule = nyse.schedule(start_date=last_date, end_date=last_date + timedelta(days=30))
        extended_schedule.index = extended_schedule.index.tz_localize(None)
        next_trading_day_index = extended_schedule.index.searchsorted(last_date, side='right')
        if next_trading_day_index < len(extended_schedule.index):
            next_trading_day = extended_schedule.index[next_trading_day_index]
        else:
            raise ValueError("No next trading day found in the extended schedule.")

    # Append the predicted regime to the data
    data.loc[next_trading_day, 'Vol_Regime'] = most_likely_regime_t1

    # For the prediction date, use the previous day's 'Adj_Close'
    data.loc[next_trading_day, 'Adj_Close'] = data['Adj_Close'].iloc[-1]

    # Recalculate moving averages and indicators
    data = add_triangular_moving_averages_and_indicators(data)

    # Define market regimes using the previous day's 'Adj_Close' and the predicted 'Vol_Regime' for next_date
    data = define_market_regimes(data)

    # Calculate exposures
    data = calculate_exposures(data)

    # Print data for the prediction date
    print("Data for prediction date:")
    print(data.loc[next_trading_day])

    # Save to SQLite database
    conn = sqlite3.connect('financial_model_test.db')

    # Ensure 'Date' is in ISO format and as a string
    data.reset_index(inplace=True)
    data.rename(columns={'index': 'Date'}, inplace=True)
    data['Date'] = pd.to_datetime(data['Date']).dt.strftime('%Y-%m-%d')

    data.to_sql('financial_data', conn, if_exists='replace', index=False)
    conn.close()

    # Send Telegram message
    telegram_messenger()


[*********************100%***********************]  1 of 1 completed


C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_6424\2620359255.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_6424\2620359255.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Index_Returns'] = data['Adj_Close'].pct_change()
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_6424\2620359255.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

                        Markov Switching Model Results                        
Dep. Variable:          Index_Returns   No. Observations:                18839
Model:               MarkovRegression   Log Likelihood               63440.697
Date:                Thu, 14 Nov 2024   AIC                        -126869.393
Time:                        17:27:11   BIC                        -126822.331
Sample:                             0   HQIC                       -126853.951
                              - 18839                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0007   5.65e-05     11.522      0.000       0.001       0.001
sigma2      3.998e-05   8.07e-07     49.543      0.0

C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_6424\2620359255.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Vol_Regime'] = result.smoothed_marginal_probabilities.idxmax(axis=1)
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_6424\2620359255.py:240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[next_trading_day, 'Vol_Regime'] = most_likely_regime_t1


Data for prediction date:
Adj_Close                                  NaN
Index_Returns                              NaN
Vol_Regime                                 0.0
30_TMA                              5831.03875
60_TMA                             5745.098881
30_Day_Indicator                       Bullish
60_Day_Indicator                       Bullish
250_TMA                            5285.491594
Market_Regime                          Unknown
Adjusted_Market_Regime    Bullish Low Variance
Portfolio_Exposure                         2.0
Name: 2024-11-15 00:00:00, dtype: object
Data used for Telegram message:
        Date Adjusted_Market_Regime  Portfolio_Exposure
0 2024-11-14   Bullish Low Variance                 2.0
1 2024-11-15   Bullish Low Variance                 2.0
Message sent successfully!


In [44]:
data = yf.download(ticker, start=start_date, end='2024-11-15')

[*********************100%***********************]  1 of 1 completed


In [45]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1950-01-03,16.660000,16.660000,16.660000,16.660000,16.660000,1260000
1950-01-04,16.850000,16.850000,16.850000,16.850000,16.850000,1890000
1950-01-05,16.930000,16.930000,16.930000,16.930000,16.930000,2550000
1950-01-06,16.980000,16.980000,16.980000,16.980000,16.980000,2010000
1950-01-09,17.080000,17.080000,17.080000,17.080000,17.080000,2520000
...,...,...,...,...,...,...
2024-11-08,5976.759766,6012.450195,5976.759766,5995.540039,5995.540039,4666740000
2024-11-11,6008.859863,6017.310059,5986.689941,6001.350098,6001.350098,4333000000
2024-11-12,6003.600098,6009.919922,5960.080078,5983.990234,5983.990234,4243400000


In [46]:
print(f"Next date for prediction: {next_date}")

Next date for prediction: 2024-11-14 00:00:00


In [47]:
datetime.today().strftime('%Y-%m-%d')

'2024-11-14'

In [49]:
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
import sqlite3
from datetime import datetime, timedelta
import requests
import pandas_market_calendars as mcal

# Function to calculate a triangular moving average
def triangular_moving_average(series, n):
    # Calculate the triangular moving average with a two-step rolling mean
    smoothed_series = series.rolling(window=(n // 2), min_periods=1).mean()
    smoothed_series = smoothed_series.rolling(window=(n // 2), min_periods=1).mean()
    return smoothed_series

# Function to add Short Term Triangular Moving Averages and Indicators
def add_triangular_moving_averages_and_indicators(data):
    # Calculate 30-day and 60-day Triangular Moving Averages
    data['30_TMA'] = triangular_moving_average(data['Adj_Close'], 30)
    data['60_TMA'] = triangular_moving_average(data['Adj_Close'], 60)
    
    # Define 30-Day and 60-Day Indicators
    data['30_Day_Indicator'] = np.where(data['Adj_Close'] > data['30_TMA'], 'Bullish', 'Bearish')
    data['60_Day_Indicator'] = np.where(data['Adj_Close'] > data['60_TMA'], 'Bullish', 'Bearish')
    
    # Shift the indicators by 1 day to use previous day's indicators for today
    data['30_Day_Indicator'] = data['30_Day_Indicator'].shift(1)
    data['60_Day_Indicator'] = data['60_Day_Indicator'].shift(1)
    
    return data

# Function to define market regimes
def define_market_regimes(data):
    # Calculate 250-day triangular moving average
    data['250_TMA'] = triangular_moving_average(data['Adj_Close'], 250)
    
    # Define the four market regimes for 250 TMA
    conditions = [
        (data['Vol_Regime'] == 1) & (data['Adj_Close'] < data['250_TMA']),
        (data['Vol_Regime'] == 1) & (data['Adj_Close'] >= data['250_TMA']),
        (data['Vol_Regime'] == 0) & (data['Adj_Close'] < data['250_TMA']),
        (data['Vol_Regime'] == 0) & (data['Adj_Close'] >= data['250_TMA']),
    ]
    choices = [
        'Bearish High Variance',
        'Bullish High Variance',
        'Bearish Low Variance',
        'Bullish Low Variance'
    ]
    
    # Specify a default value that matches the data type of choices
    data['Market_Regime'] = np.select(conditions, choices, default='Unknown')
    
    return data

# Function to calculate exposures
def calculate_exposures(data):
    # Define initial exposure based on Market_Regime
    exposure_mapping = {
        'Bullish Low Variance': 2.0,
        'Bearish Low Variance': 1.0,
        'Bullish High Variance': 1.0,
        'Bearish High Variance': 0.0
    }
    data['Portfolio_Exposure'] = data['Market_Regime'].map(exposure_mapping).fillna(1.0)  # Default exposure is 1.0 if regime is NaN
    
    # Adjust exposure based on 30-Day and 60-Day Indicators
    for index, row in data.iterrows():
        if row['Portfolio_Exposure'] == 2.0:
            if row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bearish':
                data.at[index, 'Portfolio_Exposure'] = 1.0
            elif row['30_Day_Indicator'] == 'Bullish' and row['60_Day_Indicator'] == 'Bearish':
                data.at[index, 'Portfolio_Exposure'] = 1.5
            elif row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bullish':
                data.at[index, 'Portfolio_Exposure'] = 1.5
                        
        # Adjust exposure based on 30-Day and 60-Day Indicators for exposure = 1.0 and Bearish Low Variance regime
        if row['Portfolio_Exposure'] == 1.0 and row['Market_Regime'] == 'Bearish Low Variance':
            if row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bearish':
                data.at[index, 'Portfolio_Exposure'] = 0.0
            elif row['30_Day_Indicator'] == 'Bullish' and row['60_Day_Indicator'] == 'Bearish':
                data.at[index, 'Portfolio_Exposure'] = 1.0
            elif row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bullish':
                data.at[index, 'Portfolio_Exposure'] = 1.0
                        
    return data

# Function to send Telegram message
def telegram_messenger():
    # Telegram Bot API token and Channel ID
    bot_token = '7328648943:AAH3gHyGf2xgjxBfzPd05F_7IagASgs-Dj0'
    channel_id = '-1002309744206'

    # Connect to the database
    conn = sqlite3.connect('financial_model_test.db')

    # Query the last two rows from 'Market_Regime' and 'Date' columns
    query = """
    SELECT Date, Market_Regime, Portfolio_Exposure, Prob_Regime_0, Prob_Regime_1
    FROM financial_data
    ORDER BY DATE(Date) DESC
    LIMIT 2
    """

    # Execute query and load into a DataFrame
    data = pd.read_sql_query(query, conn)

    # Close the database connection
    conn.close()

    # Convert 'Date' to datetime
    data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')

    # Sort data by Date in ascending order
    data = data.sort_values('Date').reset_index(drop=True)

    # Debugging: Print the data used for the Telegram message
    print("Data used for Telegram message:")
    print(data)

    # Initialize the message variable each time the code runs with bold header
    message = "<b>Your Daily Portfolio Exposure Update</b>\n\n"  # Reset message here
    labels = ["Today's Market Regime", "Tomorrow's Market Regime"]

    # Loop through the DataFrame and format the message
    for index, row in data.iterrows():
        # Format Date
        formatted_date = row['Date'].strftime('%m/%d/%Y').lstrip("0").replace("/0", "/")

        # Add the labeled message for each row with line breaks for better formatting
        message += f"<u>{labels[index]}</u>\n"
        message += f"<i>Date</i>: {formatted_date}\n"
        message += f"<i>Market Regime</i>: {row['Market_Regime']}\n"
        message += f"<i>Portfolio Exposure</i>: {row['Portfolio_Exposure'] * 100:.0f}%\n"
        message += f"<i>Probability Regime 0</i>: {row['Prob_Regime_0'] * 100:.2f}%\n"
        message += f"<i>Probability Regime 1</i>: {row['Prob_Regime_1'] * 100:.2f}%\n\n"

    # Telegram API URL
    api_url = f'https://api.telegram.org/bot{bot_token}/sendMessage'

    # Payload to send with HTML formatting enabled
    payload = {
        'chat_id': channel_id,
        'text': message,
        'parse_mode': 'HTML'  # Enables HTML for bold formatting
    }

    # Send the request
    response = requests.post(api_url, json=payload)

    # Check the response
    if response.status_code == 200:
        print('Message sent successfully!')
    else:
        print(f'Failed to send message. Error: {response.text}')

if __name__ == "__main__":
    # Parameters
    ticker = '^GSPC'
    start_date = '1950-01-01'
    end_date = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
    # Download data
    data = yf.download(ticker, start=start_date, end=end_date)

    # Use adjusted close price
    data = data[['Adj Close']]
    data.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)

    # Ensure that the index is timezone-naive
    data.index = data.index.tz_localize(None)

    # Calculate daily returns
    data['Index_Returns'] = data['Adj_Close'].pct_change()
    data.dropna(inplace=True)

    # Prepare returns for the model
    returns = data['Index_Returns']

    # Fit Markov Switching Model
    model = MarkovRegression(returns, k_regimes=2, trend='c', switching_variance=True)
    result = model.fit()
    print(result.summary())

    # Add regime to the data
    data['Vol_Regime'] = result.smoothed_marginal_probabilities.idxmax(axis=1)

    # Predict the next day's volatility regime
    # Extract smoothed probabilities and last known state probabilities
    smoothed_probs = result.smoothed_marginal_probabilities
    last_probs = smoothed_probs.iloc[-1].values

    # Extract transition probabilities from the model parameters
    params = result.params
    p_00 = params['p[0->0]']
    p_10 = params['p[1->0]']
    p_01 = 1 - p_00
    p_11 = 1 - p_10

    # Construct the transition matrix
    transition_matrix = np.array([
        [p_00, p_01],
        [p_10, p_11]
    ])

    # Update state probabilities to predict the next day's regime (t+1)
    state_probs_t1 = np.dot(last_probs, transition_matrix)

    # Determine the most likely regime at t+1
    regime_labels = smoothed_probs.columns.tolist()  # Should be [0, 1]
    most_likely_regime_t1 = regime_labels[np.argmax(state_probs_t1)]

    # Get the next trading day using the NYSE calendar
    nyse = mcal.get_calendar('NYSE')
    last_date = data.index[-1]

    # Create a schedule that covers the next few days
    schedule = nyse.schedule(start_date=last_date, end_date=last_date + timedelta(days=10))

    # Remove timezone information from schedule.index
    schedule.index = schedule.index.tz_localize(None)

    # Find the next trading day after 'last_date'
    next_trading_day_index = schedule.index.searchsorted(last_date, side='right')

    if next_trading_day_index < len(schedule.index):
        next_trading_day = schedule.index[next_trading_day_index]
    else:
        # If there is no next trading day in the schedule, extend the schedule and try again
        extended_schedule = nyse.schedule(start_date=last_date, end_date=last_date + timedelta(days=30))
        extended_schedule.index = extended_schedule.index.tz_localize(None)
        next_trading_day_index = extended_schedule.index.searchsorted(last_date, side='right')
        if next_trading_day_index < len(extended_schedule.index):
            next_trading_day = extended_schedule.index[next_trading_day_index]
        else:
            raise ValueError("No next trading day found in the extended schedule.")

    # Append the predicted regime and probabilities to the data
    data.loc[next_trading_day, 'Vol_Regime'] = most_likely_regime_t1
    data.loc[next_trading_day, 'Prob_Regime_0'] = state_probs_t1[0]
    data.loc[next_trading_day, 'Prob_Regime_1'] = state_probs_t1[1]

    # For the prediction date, use the previous day's 'Adj_Close'
    data.loc[next_trading_day, 'Adj_Close'] = data['Adj_Close'].iloc[-1]

    # Ensure 'Index_Returns' is NaN for the prediction date
    data.loc[next_trading_day, 'Index_Returns'] = np.nan

    # Recalculate moving averages and indicators
    data = add_triangular_moving_averages_and_indicators(data)

    # Define market regimes using the current day's 'Adj_Close' and the predicted 'Vol_Regime' for next_date
    data = define_market_regimes(data)

    # Calculate exposures
    data = calculate_exposures(data)

    # Print data for the prediction date
    print("Data for prediction date:")
    print(data.loc[next_trading_day])

    # Save to SQLite database
    conn = sqlite3.connect('financial_model_test.db')

    # Ensure 'Date' is in ISO format and as a string
    data.reset_index(inplace=True)
    data.rename(columns={'index': 'Date'}, inplace=True)
    data['Date'] = pd.to_datetime(data['Date']).dt.strftime('%Y-%m-%d')

    data.to_sql('financial_data', conn, if_exists='replace', index=False)
    conn.close()

    # Send Telegram message
    telegram_messenger()


[*********************100%***********************]  1 of 1 completed


C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_6424\3261486102.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_6424\3261486102.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Index_Returns'] = data['Adj_Close'].pct_change()
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_6424\3261486102.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

                        Markov Switching Model Results                        
Dep. Variable:          Index_Returns   No. Observations:                18839
Model:               MarkovRegression   Log Likelihood               63440.697
Date:                Thu, 14 Nov 2024   AIC                        -126869.393
Time:                        17:43:04   BIC                        -126822.331
Sample:                             0   HQIC                       -126853.951
                              - 18839                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0007   5.65e-05     11.522      0.000       0.001       0.001
sigma2      3.998e-05   8.07e-07     49.543      0.0

C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_6424\3261486102.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Vol_Regime'] = result.smoothed_marginal_probabilities.idxmax(axis=1)
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_6424\3261486102.py:239: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[next_trading_day, 'Vol_Regime'] = most_likely_regime_t1


Data for prediction date:
Adj_Close                     NaN
Index_Returns                 NaN
Vol_Regime                    0.0
Prob_Regime_0             0.90668
Prob_Regime_1             0.09332
30_TMA                 5831.03875
60_TMA                5745.098881
30_Day_Indicator          Bullish
60_Day_Indicator          Bullish
250_TMA               5285.491594
Market_Regime             Unknown
Portfolio_Exposure            1.0
Name: 2024-11-15 00:00:00, dtype: object
Data used for Telegram message:
        Date         Market_Regime  Portfolio_Exposure  Prob_Regime_0  \
0 2024-11-14  Bullish Low Variance                 2.0            NaN   
1 2024-11-15               Unknown                 1.0        0.90668   

   Prob_Regime_1  
0            NaN  
1        0.09332  
Message sent successfully!
